# Projeto 1 - Ciência dos Dados

Nome: Lorena Barbosa Antunes Da Silva

Nome: José Edson Mendonça Ribeiro Lima Araújo

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [5]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
!pip install emoji
import emoji

Processing c:\users\loren\appdata\local\pip\cache\wheels\e9\d5\46\33c9101a710eb267fbaf8572c96fce5b12702e62bb5fa40592\emoji-1.5.0-py3-none-any.whl


In [6]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\loren\Repositorios\CDados_Projeto1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [7]:
filename = 'Peaky Blinders.xlsx'

In [8]:
train = pd.read_excel(filename)
linha=np.arange(0,300,1)
train=train.iloc[linha,[0,1]]
train.head(5)
#train.tail()

,Treinamento,Classificação
0,peaky fucking blinders foi a melhor coisa que ...,1.0
1,@limawt1 peaky blinders está vindo em breve!! ...,0.0
2,gente cancela ele me seguiu de outro instagram...,0.0
3,até eu que não sou fumante fico com vontade de...,0.0
4,tou a curtir bue de peaky blinders,1.0


In [9]:
test = pd.read_excel(filename, sheet_name = 'Teste')
linha=np.arange(0,200,1)
test=test.iloc[linha,[0,1]]
test.head(5)
test.tail(5)

,Teste,Classificação
195,viciada em peaky blinders 🖤,1.0
196,@pedropossobom peaky blinders mastermind \nmei...,1.0
197,@italoverdose peaky blinders está vindo em bre...,0.0
198,"a gente fuma tbem, somos os peaky blinders htt...",0.0
199,o lado bom de tá com covid e que eu tô maraton...,1.0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

Produto: Série Peaky Blinders, consideramos relevante qualquer comentário positivo ou negativo, relacionados a série mostrando que estão assistindo ou indicando. Exemplo: "Não curti assistir Peaky Blinders", "@user1 vc precisa assistir peaky blinders", "acabei de maratonar essa serie peaky blinders".

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [10]:
from IPython.display import display
pd.options.display.max_rows = 13


import re 



def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    limpa_emoji=' '.join(emoji.get_emoji_regexp().split(text))
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', limpa_emoji)
    
    return text_subbed

In [14]:
comentarios_relevantes=[]
comentarios_irrelevantes=[]
for i in range(0,300,1):
    comentario=cleanup(train.Treinamento[i]).lower().split()
    if train.Classificação[i]==1:
        comentarios_relevantes.append(comentario)
    elif train.Classificação[i]==0:
        comentarios_irrelevantes.append(comentario)

In [53]:
l_irrelevantes=[]
for comentario in comentarios_irrelevantes:
    lista_remocao=[]
    for palavra in comentario:
        if  palavra[0]=='@' or 'htt' == palavra[0:3]:
            lista_remocao.append(palavra)
    for palavra in lista_remocao:
        comentario.remove(palavra)

for comentario in comentarios_irrelevantes:
    for palavra in comentario:
        l_irrelevantes.append(palavra)

for comentario in comentarios_relevantes:
    lista_remocao=[]
    for palavra in comentario:
        if  palavra[0]=='@' or 'htt' == palavra[0:3]:
            lista_remocao.append(palavra)
    for palavra in lista_remocao:
        comentario.remove(palavra)

In [57]:
s_relevantes = pd.Series(comentarios_relevantes)
s_l_irrelevantes=  pd.Series(l_irrelevantes)
s_l_irrelevantes.value_counts().to_frame()

,0
peaky,149
blinders,148
o,77
da,71
de,67
...,...
vive,1
papel,1
maluca,1
varias,1


In [27]:
s_irrelevantes = pd.Series(comentarios_irrelevantes)

In [49]:
tab_relevantes = s_relevantes.value_counts().to_frame()
tab_relevantes

,0
"[viciei, em, peaky, blinders]",1
"[voltei, a, assistir, peaky, blinders, pra, roubar, a, personalidade, fria, e, calculista]",1
"[vontade, de, assistir, peaky, blinders, dnv]",1
"[amo, você, lt3, eu, tô, aproveitando, meus, últimos, momentos, de, férias, pra, ver, seriados, comecei, peaky, blinders, esses, dias, e, fiquei, viciadissima]",1
"[que, vontade, de, ver, peaky, blinders, inteiro, dnv…, mesmo, já, tendo, visto, 3, vzs]",1
...,...
"[terminei, de, ver, peaky, blinders, e, pqp, q, série, foda]",1
"[quem, diria, eu, viciei, em, peaky, blinders]",1
"[botei, peaky, blinders, só, pra, fazer, barulho, em, casa, agora, to, aqui, ignorando, todas, as, minhas, obrigações, pra, terminar, só, mais, esse, episódio]",1
"[lá, vou, eu, assistir, peaky, blinders, novamente, já, me, perdi, nas, contas]",1


In [30]:
tab_irrelevantes = s_irrelevantes.value_counts()

In [37]:
total= comentarios_relevantes+comentarios_irrelevantes

In [38]:
s_total= pd.Series(total)

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [39]:
l_test = []
l_test_relevantes=[]
l_test_irrelevantes=[]

for i in range(0,200,1):
    l_test=cleanup(test.Teste[i]).lower().split()
    
    if test.Classificação[i]==1:
        l_test_relevantes.append(l_test)
        
    elif test.Classificação[i]==0:
        l_test_irrelevantes.append(l_test)

In [41]:
P_relevante = len(comentarios_relevantes)/len(total)
P_irrelevante = len(comentarios_irrelevantes)/len(total)

In [ ]:
Pro

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**